- we don't have a left and right state so can eliminate all the boundary if statement stuff 
    - delete all the left states 
        - then delete all "r" in the right state names 
- U==q?? 
- par_getd gets the values from the input file in the "problem" block
-  Structs in C are like objects/classes but without methods or public/private specifications. "Members" are just instance variables.
- look into athena.h for how the object structs are defined 
- Difference between ConsS and Cons1Ds
    - except Cons1DS doesn't contain $B_x$, not relevant for our case here anyways, but why?/

- Before trying to setup the torus first run the problem and see if you could set up a constant background and get it running 

- it looks like vfr and vfl is a quantity between [0,1] that sorta tells you how much volume the fluid occupy of percentage of the cell. This is determined by a series of if else cases that figures out whtether it is half, more thna half of fully in the left or right cell, so then you use this to do volume averaging . For now since I took out that whole if else I'm just going to assume that vf=1.0, so its not necessary to be in here. But for now I'm going to put vf in as a placeholder so that if I really need  to figure out the boundary cases later I could. 


###Input File
- using athinput.shk_cloud as a template (doens't really matter) 
- nscalars 	# 	add # passively advected scalars (default is 0)
- inthis case it should be zero in the flag so default


- does the gas need to be confined in the torus shape for all time or only during the initial conditions ? (since PPI don't show up unless reflecting boundary at inner and outer boundaries 

##Running the code
- place ppi.c to ./athena/src/prob
- putting the input file just in athena directory anywhere
```
./configure --with-problem=ppi --with-gas=hydro
make all
```


Errors to be fixed: 
```
problem.c: In function ‘problem’:
problem.c:65: error: too few arguments to function ‘Prim1D_to_Cons1D’
problem.c: At top level:
problem.c:96: error: expected identifier or ‘(’ before ‘return’
problem.c:97: error: expected identifier or ‘(’ before ‘}’ token
problem.c: In function ‘problem_read_restart’:
problem.c:111: error: ‘Omega_0’ undeclared (first use in this function)
problem.c:111: error: (Each undeclared identifier is reported only once
problem.c:111: error: for each function it appears in.)
problem.c:112: error: ‘hst_rho_Vx_dVy’ undeclared (first use in this function)
problem.c:113: error: ‘hst_rho_dVy2’ undeclared (first use in this function)
problem.c: At top level:
problem.c:117: error: redefinition of ‘problem_read_restart’
problem.c:109: note: previous definition of ‘problem_read_restart’ was here
make[1]: *** [problem.o] Error 1
make[1]: Leaving directory `/u/dorislee/Programs/athena4.2/src'
make: *** [compile] Error 2
```


Fixed everything but this is weird: 
```
    problem.c:113: error: redefinition of ‘problem_read_restart’
problem.c:108: note: previous definition of ‘problem_read_restart’ was here
make[1]: *** [problem.o] Error 1
make[1]: Leaving directory `/u/dorislee/Programs/athena4.2/src'
make: *** [compile] Error 2
```
But I am not defining any structs in the code at all! (so none of the  SO solutions apply!)
- after delving this for a long time, I decided to compare this with collapse3d.c inside src/prob/ and compared this line by line to it. I noticed that my comment formatting was kind of weird then I realized that I never closed my second comment. Since there was another comment right under it, it didn't complain to me and turn everything blue, that's why I missed it. After putting that end comment */ back in it ``make all`` sucessfully now.

```
$ athena -i athinput.ppi 
Simulation started on Tue Jun 23 09:02:04 2015

### Fatal error: par_gets: Par "x3min" not found in Block "domain1"
```
I dont understand why I need to define things for x3 if I am only running this in 2D ?
But let's do it anyways min max as 0.0,0.0


Kept getting fatal error do variable not found but there is no d0 in my c code file!!!
turns out I was running --with-problem=collapse3d rather than --with-problem=ppi accidentally!

```
problem.c:101: error: redefinition of ‘problem_read_restart’
problem.c:96: note: previous definition of ‘problem_read_restart’ was here
make[1]: *** [problem.o] Error 1
make[1]: Leaving directory `/u/dorislee/Programs/athena4.2/src'
```
giving me this error again even though the comment is fixed

- I forgot to write a null function for problem_write_restart
- stil same error
- The real reason is that I defined problem_read_restart twice!! So the dual definition was what threw this off!
- Ok now everything compiles correctly and ``athena -i athinput.ppi`` runs sucessfully!